In [ ]:
from openai import OpenAI
import os
import re
from typing import List, Dict

# API
# Iput your DeepSeek API 
# Refer to official documentation: https://www.deepseek.com/
API_KEY = os.getenv("DEEPSEEK_API_KEY", "")  # Your API 

# 原始数据处理
def parse_dialogue(file_path: str) -> List[Dict[str, str]]:
    """解析对话文件（跳过第一行标题，忽略时间戳）"""
    dialogues = []
    current_role = None
    is_first_line = True  # 标记是否为第一行
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # 跳过空行
                continue

            # 跳过第一行（标题行）
            if is_first_line:
                is_first_line = False
                continue

            # 检测角色行（匹配 "说话人X" 或 "[角色X]" 格式）
            role_match = re.match(r'^(说话人\d+|\[角色\d+\])\s*(.*)', line)
            if role_match:
                current_role = role_match.group(1)  # 提取角色
                content = role_match.group(2)       # 提取内容

                # 移除内容中的时间戳（例如 "00:00 内容"）
                content = re.sub(r'^\d+:\d+\s*', '', content)

                if content:
                    dialogues.append({
                        "role": current_role,  # 不包含时间戳
                        "content": content
                    })
                continue
                
            # 处理延续内容
            if current_role and dialogues:
                # 移除内容中的时间戳（例如 "00:00 内容"）
                line = re.sub(r'^\d+:\d+\s*', '', line)

                # 追加到上一行内容
                if dialogues[-1]["role"] == current_role:
                    dialogues[-1]["content"] += "\n" + line
                else:
                    dialogues.append({
                        "role": current_role,  # 不包含时间戳
                        "content": line
                    })
            else:
                # 移除内容中的时间戳（例如 "00:00 内容"）
                line = re.sub(r'^\d+:\d+\s*', '', line)
                dialogues.append({
                    "role": "unknown",
                    "content": line
                })
    
    return dialogues

def generate_medical_dialogue_summary(input_file: str) -> str:
    """生成规范化医患对话记录"""
    try:
        # 初始化Deepseek客户端
        client = OpenAI(
            api_key=API_KEY,
            base_url="https://api.deepseek.com"
        )
        
        # 解析并预处理对话
        print("*********正在读取原始医患对话*********")
        raw_data = parse_dialogue(input_file)
        
        # 将对话转换为易读文本
        dialogue_text = "\n".join(
            [f"[{item['role']}] {item['content']}" for item in raw_data]
        )
        
        # 构建专业prompt
        system_prompt = """您是一位专业的医疗对话数据处理专家，需要执行以下操作：
        
        我将给你一个医患对话的转写文稿文件，你需要对其进行处理，
        1: 在每句话前面加入患者和医生的身份标签。请注意，文稿中已经预先进行了说话人识别，但无法判断是医生还是患者说的，并且说话人的识别也可能存在着误判; 注意文稿中可能出现多名身份标签，请注意修正为只有患者和医生；
        2: 纠正转写文稿中的错别字，音频转写文本时出现的各类错误，以及不通顺语句、不合逻辑不合上下文的话语，并且也要保证其中医学术语的正确性;
        3: 对文稿中已出现的人名（或其他隐私内容）进行严格匿名化处理：用XXX进行替代;
        4: 进行规范化、格式化的输出;
        5: 进行对话主题的判断，并且在每一段对话的最开始写出你判断的对话主题;
        6: 不带省略地、全部输出你修改后的文稿，并且在生成文稿后，附带一个对文稿修改部分对比的表格。
        请使用简体中文回答。"""

        user_prompt = f"""请处理以下医患对话：
        
        原始对话记录：
        {dialogue_text}

        请严格按以下格式返回：
        ### 对话主题
        [你的判断]

        ### 规范化记录
        [你的修改结果]

        ### 修改对照表
        | 原句 | 修改后 |
        |------|--------|
        [对比表格]"""

        # 调用API
        print("\n*********正在生成规范化记录*********")
        response = client.chat.completions.create(
            model="deepseek-reasoner", # 调用推理模型  deepseek-reasoner
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=1.0,  # 降低随机性
            max_tokens=4000   # 根据需求调整
        )
        
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        print(f"处理失败: {str(e)}")
        return "生成失败，请检查输入文件或API配置"

# 使用示例
if __name__ == "__main__":
    result = generate_medical_dialogue_summary(
        input_file="./transcription_result_with_roles.txt")  # Change your input 
    print("\n*********结果如下*********")
    print(result)
    # 保存为 TXT 文件
    with open('Result.txt', 'w', encoding='utf-8') as f:
        if isinstance(result, list):
            f.write("\n".join(result))  # 如果 result 是列表，将其转换为字符串
        else:
            f.write(result)  # 如果 result 是字符串，直接写入
    print("\n*********结果已经保存*********")